In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat, Swarm
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [2]:
# google_search_tool = FunctionTool(
#     google_search, description="Search Google for information, returns results with a snippet and body content"
# )

from tools import arxiv_search, save_report
from web_search import bing_search

arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)
bing_search_tool = FunctionTool(
    bing_search, description="Search Bing for information, returns results with a snippet and body content"
)
save_report_tool = FunctionTool(
    save_report, description="Save report to file"
)

In [3]:
# google_search_agent = AssistantAgent(
#     name="Google_Search_Agent",
#     tools=[google_search_tool],
#     model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
#     description="An agent that can search Google for information, returns results with a snippet and body content",
#     system_message="You are a helpful AI assistant. Solve tasks using your tools.",
# )
from autogen_core.models import ModelFamily

model_client = OpenAIChatCompletionClient(
    # model="qwen2.5:14b",
    model="qwen2.5:7b",
    # model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

search_agent = AssistantAgent(
    name="Bing_Search_Agent",
    model_client=model_client,
    tools=[bing_search_tool],
    description="Search Bing for information, returns top 2 results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    # model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    model_client=model_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query to the arxiv_search_tool tool that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    # model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    model_client=model_client,
    tools=[save_report],
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references. You MUST save the final report to file using the save_report tool. Your response should end with the word 'TERMINATE'",
)

In [4]:
termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)

# selector_prompt = """Select an agent to perform task.

# {roles}

# Current conversation context:
# {history}

# Read the above conversation, then select an agent from {participants} to perform the next task.
# Make sure the planner agent has assigned tasks before other agents start working.
# Only select one agent.
# """
# team = SelectorGroupChat(
#     participants=[arxiv_search_agent, search_agent, report_agent], 
#     termination_condition=termination,
#     selector_prompt=selector_prompt,
#     model_client=model_client,
#     allow_repeated_speaker=True,
# )

# export OLLAMA_CONTEXT_LENGTH=128000 and run ollama serve
# as the web page content might be huge, the context need to be increased.

await Console(
    team.run_stream(
        task="Write a literature review on no code tools for building multi agent ai systems",
    ),
    output_stats=True,
)

---------- user ----------
Write a literature review on no code tools for building multi agent ai systems
---------- Bing_Search_Agent ----------
[FunctionCall(id='call_iwutbr3q', arguments='{"max_chars":10000,"query":"no code tools for building multi agent ai systems"}', name='bing_search')]
[Prompt tokens: 184, Completion tokens: 99]
search results: 9
---------- Bing_Search_Agent ----------
[FunctionExecutionResult(content='[{\'title\': \'AutoGen Studio: A No-Code Developer Tool for Building and …\', \'link\': \'https://arxiv.org/abs/2408.15247\', \'snippet\': \'2024年8月9日\\u2002·\\u2002To address this challenge, we present AUTOGEN STUDIO, a no-code developer tool for rapidly prototyping, debugging, and evaluating multi-agent workflows built upon the …\', \'body\': "[2408.15247] AutoGen Studio: A No-Code Developer Tool for Building and Debugging Multi-Agent Systems Skip to main content We gratefully acknowledge support from the Simons Foundation, member institutions , and all contribu

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a literature review on no code tools for building multi agent ai systems', type='TextMessage'), ToolCallRequestEvent(source='Bing_Search_Agent', models_usage=RequestUsage(prompt_tokens=184, completion_tokens=99), metadata={}, content=[FunctionCall(id='call_iwutbr3q', arguments='{"max_chars":10000,"query":"no code tools for building multi agent ai systems"}', name='bing_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Bing_Search_Agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='[{\'title\': \'AutoGen Studio: A No-Code Developer Tool for Building and …\', \'link\': \'https://arxiv.org/abs/2408.15247\', \'snippet\': \'2024年8月9日\\u2002·\\u2002To address this challenge, we present AUTOGEN STUDIO, a no-code developer tool for rapidly prototyping, debugging, and evaluating multi-agent workflows built upon the …\', \'body\': "[2408.15247] AutoGe